# LLMs Q&A Challenge 💬

## 🎯 Objective

The goal is to compare the behavior of **multiple Large Language Models (LLMs)** under the same prompts, reusing code and concepts introduced in Week 1.

In particular, four models are compared: gpt-5-nano, llama3.2, gemini-2.5-pro, and deepseek-r1:1.5b

The “challenge” consists of asking each model **the same set of questions**, including:
- deliberately **nonsensical or ambiguous** prompts,
- **metalinguistic** requests (e.g., word counting),
- **abstract descriptive** questions,
- **self-reflective** questions about the model’s own capabilities and limitations.

The objective is not to identify a “winner,” but rather to observe **qualitative differences** in how models interpret, handle, and respond to the same input.

---

## ⚙️ Requirements

To run the notebook correctly, the following are required:

- The following API keys (when required by the selected models):
  - OpenAI API key
  - Google Gemini API key
- Ollama installed and running locally for locally executed models
  (e.g., `ollama serve` from the terminal)

API keys must be loaded via a `.env` file, as shown in the previous notebooks.

---

## 🔍 Observations

When analyzing the responses, it is important to keep in mind several key aspects:

- **Same prompt ≠ same context**
  Even with identical textual input, models operate under different conditions:
  tokenizers, implicit system prompts, safety policies, and generation strategies are not uniform. These factors directly affect the structure and content of the responses.

- **Nonsense questions and handling of absurdity**
  Intentionally meaningless prompts make it possible to observe whether a model:
  - attempts to “invent” an answer anyway (hallucination),
  - explicitly signals that the question has no meaning,
  - freely reinterprets or reformulates the prompt.

- **Instability of metalinguistic requests**
  Questions such as *“How many words are there in your answer to this question?”* are inherently unstable:
  word counts may vary depending on punctuation, spacing, reformulations, and even on different versions of the generated response.

These observations help illustrate that LLM behavior does not depend solely on model “power,” but also on **architectural and design choices** that are often invisible to the end user.

---

## 🌐 External reference

For those interested in seeing language models tested in a more playful setting, there is also an interesting and entertaining project available at:

👉 https://edwarddonner.com/outsmart/

It is a game of **strategy and diplomacy** in which different LLMs compete directly, revealing how strategies, alliances, and communication failures emerge depending on the model being used. This provides a compelling example of LLM comparison beyond a purely technical context.


In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import time

In [ ]:
load_dotenv(override=True)

# Gemini: gemini-2.5-pro
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
google_api_key = os.getenv("GOOGLE_API_KEY")
assert google_api_key, "Missing GOOGLE_API_KEY in .env"

#OpenAI: gpt-5-nano
OPENAI_BASE_URL = "https://api.openai.com/v1/"
open_api_key = os.getenv('OPENAI_API_KEY')
assert open_api_key, "Missing OPENAI_API_KEY in .env"

#Ollama: llama3.2 and deepseek-r1:1.5b
OLLAMA_BASE_URL="http://localhost:11434/v1"
ollama_api_key="ollama"

In [ ]:
QUESTIONS = [
    "how many rainbows does it take to jump from Hawaii to 17?",
    "how many words are there in your answer to this question?",
    "how would you describe the color red to someone who's never been able to see?",
    "Compared with other Frontier LLMs, what kinds of questions are you best at answering, "
     "and what kinds of questions do you find most challenging? Which other LLM has capabilities that complement yours?"
]

In [ ]:
MODELS = [
    {"model_name":"gemini-2.5-pro",
     "base_url":GEMINI_BASE_URL,
     "api_key":google_api_key},
    {"model_name":"llama3.2",
     "base_url":OLLAMA_BASE_URL,
     "api_key":ollama_api_key},
    {"model_name":"deepseek-r1:1.5b",
     "base_url":OLLAMA_BASE_URL,
     "api_key":ollama_api_key},
    {"model_name":"gpt-5-nano",
     "base_url": OPENAI_BASE_URL,
     "api_key": open_api_key}]

In [ ]:
# Let's try!
for model in MODELS:
    print(model["model_name"])
    client = OpenAI(base_url=model["base_url"], api_key=model["api_key"])
    response = client.chat.completions.create(model=model["model_name"], messages=[{"role":"user", "content":"hi"}])
    print(response.choices[0].message.content)

In [ ]:
# Function to end a single question to a language model and return its response.
def response_instance(model, question):
    client = OpenAI(base_url=model["base_url"], api_key=model["api_key"])
    t0 = time.time()
    response = client.chat.completions.create(model=model["model_name"], messages=[{"role":"user", "content":question}])
    latency = time.time() - t0
    return response, latency

# Function to count the number of words in a text string.
def word_count(s: str) -> int:
    return len([w for w in s.strip().split() if w])

In [ ]:
# Function to run the multi-model question-answering challenge.
def run_challenge(models, questions) -> pd.DataFrame:
    rows = []
    for q, question in enumerate(questions, start=1):
        for model in models:
            try:
                response, latency = response_instance(model, question)
                text = response.choices[0].message.content
                rows.append({
                    "q_id": q,
                    "question": question,
                    "model": model["model_name"],
                    "latency_s": latency,
                    "word_count": word_count(text),
                    "answer": text,
                })
            except Exception as e:
                rows.append({
                    "q_id": q,
                    "question": question,
                    "model": model["model_name"],
                    "latency_s": None,
                    "word_count": None,
                    "answer": f"[ERROR] {type(e).__name__}: {e}",
                })
    return pd.DataFrame(rows)

In [ ]:
# Run the challange
df = run_challenge(MODELS, QUESTIONS)

In [ ]:
# Create a compact summary table with basic quantitative metadata
summary = df[["q_id", "model", "latency_s", "word_count"]].sort_values(["q_id", "model"])
summary

In [ ]:
# Function to display model responses for a single question in a readable, ordered format.
def show_answers(df: pd.DataFrame, q_id: int):
    block = df[df["q_id"] == q_id].sort_values("model")
    print(f"\nQ{q_id}: {block.iloc[0]['question']}\n" + "-"*80)
    for _, r in block.iterrows():
        print(f"\n### {r['model']} | {r['latency_s']:.2f}s | {r['word_count']} words\n")
        print(r["answer"])

show_answers(df, 1)

### Final note

This experiment highlights how Large Language Models differ not only in
knowledge and reasoning style, but also in how they interpret ill-defined,
self-referential, or nonsensical prompts. Even simple metrics such as word
count or response length can vary significantly, reminding us that LLM
behavior is deeply shaped by architectural choices, training data, and
inference-time policies.